# Loading Responses

In [1]:
options(warn=1)
if(!require('lme4')) {
    install.packages('lme4', repos='https://cloud.r-project.org')
    library('lme4')
}
if(!require('readr')) {
    install.packages('readr', repos='https://cloud.r-project.org')
    library('readr')
}
if(!require('marginaleffects')) {
    install.packages('marginaleffects', repos='https://cloud.r-project.org')
    library('marginaleffects')
}
if(!require('emmeans')) {
    install.packages('emmeans', repos='https://cloud.r-project.org')
    library('emmeans')
}
if(!require('car')) {
    install.packages('car', repos='https://cloud.r-project.org')
    library('car')
}
df <- read_csv("gaze-analysis/result_1743457603_20250506_20250506F.csv", na = "empty", col_select = c("Accuracy", "Group", "GroupKind", "Angle", "Proximity", "n_candidates", "Actor", "Candidates", "Stimulus_ID", "Prompt_ID", "Participant_ID", "list_id", "Run_ID", "Part"), col_types = cols(
    Accuracy = col_logical(),
    Group = col_factor(),
    GroupKind = col_factor(),
    Angle = col_factor(),
    Proximity = col_integer(),
    n_candidates = col_integer(),
    Actor = col_factor(c('X', 'Y')),
    Candidates = col_factor(),
    Stimulus_ID = col_factor(),
    Prompt_ID = col_factor(),
    Participant_ID = col_factor(),
    list_id = col_factor(),
    Run_ID = col_character(),
    Part = col_character(),
),show_col_types = TRUE)
df <- df[df$Group == "gemini-1.5-pro" & df$list_id != -1 & df$Part != "p0", ]
df$offset <- log(1/df$n_candidates / (1 - 1/df$n_candidates))
#df$Proximity <- df$Proximity - mean(df$Proximity)
#df$n_candidates <- df$n_candidates - mean(df$n_candidates)

Loading required package: lme4

Loading required package: Matrix

Loading required package: readr

Loading required package: marginaleffects

Loading required package: emmeans

Welcome to emmeans.
Caution: You lose important information if you filter this package's results.
See '? untidy'

Loading required package: car

Loading required package: carData

Rows: 156780 Columns: 14
── Column specification ────────────────────────────────────────────────────────────────
Delimiter: ","
chr (2): Run_ID, Part
int (2): Proximity, n_candidates
lgl (1): Accuracy
fct (9): Stimulus_ID, Prompt_ID, Participant_ID, Group, GroupKind, Angle, Ac...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


# Fitting models

In [2]:
model0 <- glmer(Accuracy ~ Angle + scale(Proximity, scale=FALSE) + scale(n_candidates, scale=FALSE) + Actor + (1|Actor:Candidates) + (1|Stimulus_ID) + (1|Prompt_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)
model0

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.0488344 (tol = 0.002, component 1)”


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Accuracy ~ Angle + scale(Proximity, scale = FALSE) + scale(n_candidates,  
    scale = FALSE) + Actor + (1 | Actor:Candidates) + (1 | Stimulus_ID) +  
    (1 | Prompt_ID)
   Data: df
 Offset: offset
      AIC       BIC    logLik  deviance  df.resid 
 7013.281  7077.226 -3497.640  6995.281      8991 
Random effects:
 Groups           Name        Std.Dev.
 Stimulus_ID      (Intercept) 4.044967
 Actor:Candidates (Intercept) 0.004815
 Prompt_ID        (Intercept) 0.156186
Number of obs: 9000, groups:  
Stimulus_ID, 900; Actor:Candidates, 18; Prompt_ID, 12
Fixed Effects:
                       (Intercept)                           Angleleft  
                            0.5368                             -0.4367  
                        Angleright     scale(Proximity, scale = FALSE)  
                           -0.2481                             -0.7262  
sc

In [3]:
model1 <- glmer(Accuracy ~ Angle + scale(Proximity, scale=FALSE) + scale(n_candidates, scale=FALSE) + Actor + (1|Stimulus_ID) + (1|Prompt_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)
model1

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Accuracy ~ Angle + scale(Proximity, scale = FALSE) + scale(n_candidates,  
    scale = FALSE) + Actor + (1 | Stimulus_ID) + (1 | Prompt_ID)
   Data: df
 Offset: offset
      AIC       BIC    logLik  deviance  df.resid 
 7011.277  7068.117 -3497.639  6995.277      8992 
Random effects:
 Groups      Name        Std.Dev.
 Stimulus_ID (Intercept) 4.043   
 Prompt_ID   (Intercept) 0.157   
Number of obs: 9000, groups:  Stimulus_ID, 900; Prompt_ID, 12
Fixed Effects:
                       (Intercept)                           Angleleft  
                            0.5441                             -0.4442  
                        Angleright     scale(Proximity, scale = FALSE)  
                           -0.2524                             -0.7219  
scale(n_candidates, scale = FALSE)                              ActorY  
                           -0.9059   

In [4]:
model2 <- glmer(Accuracy ~ Angle + scale(Proximity, scale=FALSE) + scale(n_candidates, scale=FALSE) + Actor + (1|Actor:Candidates) + (1|Stimulus_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)
model2

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Accuracy ~ Angle + scale(Proximity, scale = FALSE) + scale(n_candidates,  
    scale = FALSE) + Actor + (1 | Actor:Candidates) + (1 | Stimulus_ID)
   Data: df
 Offset: offset
      AIC       BIC    logLik  deviance  df.resid 
 7017.499  7074.339 -3500.750  7001.499      8992 
Random effects:
 Groups           Name        Std.Dev. 
 Stimulus_ID      (Intercept) 4.0269663
 Actor:Candidates (Intercept) 0.0001601
Number of obs: 9000, groups:  Stimulus_ID, 900; Actor:Candidates, 18
Fixed Effects:
                       (Intercept)                           Angleleft  
                            0.5475                             -0.4455  
                        Angleright     scale(Proximity, scale = FALSE)  
                           -0.2508                             -0.7178  
scale(n_candidates, scale = FALSE)                              ActorY  
     

In [5]:
model3 <- glmer(Accuracy ~ Angle + scale(Proximity, scale=FALSE) + scale(n_candidates, scale=FALSE) + Actor + (1|Stimulus_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)
model3

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Accuracy ~ Angle + scale(Proximity, scale = FALSE) + scale(n_candidates,  
    scale = FALSE) + Actor + (1 | Stimulus_ID)
   Data: df
 Offset: offset
      AIC       BIC    logLik  deviance  df.resid 
 7015.499  7065.234 -3500.750  7001.499      8993 
Random effects:
 Groups      Name        Std.Dev.
 Stimulus_ID (Intercept) 4.027   
Number of obs: 9000, groups:  Stimulus_ID, 900
Fixed Effects:
                       (Intercept)                           Angleleft  
                            0.5473                             -0.4451  
                        Angleright     scale(Proximity, scale = FALSE)  
                           -0.2507                             -0.7179  
scale(n_candidates, scale = FALSE)                              ActorY  
                           -0.8970                             -0.1651  

In [6]:
model3_5 <- glmer(Accuracy ~ Angle + scale(Proximity, scale=FALSE) + scale(n_candidates, scale=FALSE) + Actor + (1|Actor:Candidates), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)
model3_5

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Accuracy ~ Angle + scale(Proximity, scale = FALSE) + scale(n_candidates,  
    scale = FALSE) + Actor + (1 | Actor:Candidates)
   Data: df
 Offset: offset
      AIC       BIC    logLik  deviance  df.resid 
12024.321 12074.056 -6005.161 12010.321      8993 
Random effects:
 Groups           Name        Std.Dev.
 Actor:Candidates (Intercept) 0.1981  
Number of obs: 9000, groups:  Actor:Candidates, 18
Fixed Effects:
                       (Intercept)                           Angleleft  
                           0.47627                            -0.26027  
                        Angleright     scale(Proximity, scale = FALSE)  
                          -0.18684                            -0.20687  
scale(n_candidates, scale = FALSE)                              ActorY  
                           0.10384                            -0.05227  

In [7]:
model4 <- glm(Accuracy ~ Angle * scale(Proximity, scale=FALSE) * scale(n_candidates, scale=FALSE) * Actor, 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)
model4


Call:  glm(formula = Accuracy ~ Angle * scale(Proximity, scale = FALSE) * 
    scale(n_candidates, scale = FALSE) * Actor, family = binomial(link = "logit"), 
    data = df, offset = offset)

Coefficients:
                                                                         (Intercept)  
                                                                           0.5436877  
                                                                           Angleleft  
                                                                          -0.3251485  
                                                                          Angleright  
                                                                          -0.2606240  
                                                     scale(Proximity, scale = FALSE)  
                                                                          -0.3674851  
                                                  scale(n_candidates, scale = FALSE)  
          

In [8]:
model5 <- glmer(Accuracy ~ Angle + scale(Proximity, scale=FALSE) * Actor + scale(n_candidates, scale=FALSE) + (1|Stimulus_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)
model5

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: Accuracy ~ Angle + scale(Proximity, scale = FALSE) * Actor +  
    scale(n_candidates, scale = FALSE) + (1 | Stimulus_ID)
   Data: df
 Offset: offset
      AIC       BIC    logLik  deviance  df.resid 
 7014.502  7071.342 -3499.251  6998.502      8992 
Random effects:
 Groups      Name        Std.Dev.
 Stimulus_ID (Intercept) 4.024   
Number of obs: 9000, groups:  Stimulus_ID, 900
Fixed Effects:
                           (Intercept)                               Angleleft  
                                0.5490                                 -0.4454  
                            Angleright         scale(Proximity, scale = FALSE)  
                               -0.2559                                 -1.0186  
                                ActorY      scale(n_candidates, scale = FALSE)  
                               -0.1653                           

# Model selection

In [9]:
anova(model1, model2)

,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
model1,8,7011.277,7068.117,-3497.639,6995.277,NA,NA,NA
model2,8,7017.500,7074.339,-3500.750,7001.500,0,0,NA


In [10]:
anova(model1, model3)

,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
model3,7,7015.499,7065.234,-3500.750,7001.499,NA,NA,NA
model1,8,7011.277,7068.117,-3497.639,6995.277,6.222326,1,0.01261493


In [11]:
anova(model3, model3_5)

,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
model3,7,7015.499,7065.234,-3500.750,7001.499,NA,NA,NA
model3_5,7,12024.321,12074.056,-6005.161,12010.321,0,0,NA


In [12]:
anova(model3, model4)

,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
model3,7,7015.499,7065.234,-3500.750,7001.499,NA,NA,NA
model4,24,12007.389,12177.908,-5979.694,11959.389,0,17,1


In [13]:
anova(model1, model5)

,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
model1,8,7011.277,7068.117,-3497.639,6995.277,NA,NA,NA
model5,8,7014.502,7071.342,-3499.251,6998.502,0,0,NA


In [14]:
model <- model1

# Interpretation

In [15]:
summary(model)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Accuracy ~ Angle + scale(Proximity, scale = FALSE) + scale(n_candidates,  
    scale = FALSE) + Actor + (1 | Stimulus_ID) + (1 | Prompt_ID)
   Data: df
 Offset: offset

     AIC      BIC   logLik deviance df.resid 
  7011.3   7068.1  -3497.6   6995.3     8992 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.1910 -0.1774 -0.0992  0.1699  3.1835 

Random effects:
 Groups      Name        Variance Std.Dev.
 Stimulus_ID (Intercept) 16.34973 4.043   
 Prompt_ID   (Intercept)  0.02464 0.157   
Number of obs: 9000, groups:  Stimulus_ID, 900; Prompt_ID, 12

Fixed effects:
                                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)                          0.5441     0.3072   1.771 0.076559 .  
Angleleft                           -0.4442     0.3813  -1.165 0.244009    
Angleright                          -0.2524     0.38

In [16]:
vc_list <- lapply(VarCorr(model), function(x) attr(x, "stddev")^2)
vc <- sum(unlist(vc_list))
effect_size_denom_t <- sqrt(vc + pi^2/3)

res <- summary(model)$coefficients
res <- as.data.frame(res)
res <- cbind(term = rownames(res), res)
rownames(res) <- NULL
res$effect_size_denom <- effect_size_denom_t
res$effect_size <- res$Estimate / effect_size_denom_t
res

term,Estimate,Std. Error,z value,Pr(>|z|),effect_size_denom,effect_size
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.5441380,0.3072468,1.7710128,7.655858e-02,4.434438,0.12270733
Angleleft,-0.4442305,0.3813060,-1.1650237,2.440094e-01,4.434438,-0.10017742
Angleright,-0.2523914,0.3824899,-0.6598642,5.093410e-01,4.434438,-0.05691622
"scale(Proximity, scale = FALSE)",-0.7219165,0.1929979,-3.7405405,1.836249e-04,4.434438,-0.16279775
"scale(n_candidates, scale = FALSE)",-0.9059379,0.2076662,-4.3624708,1.286017e-05,4.434438,-0.20429602
ActorY,-0.1659441,0.3146023,-0.5274727,5.978654e-01,4.434438,-0.03742169


In [17]:
avg_predictions(model, by="Angle", hypothesis = ~ pairwise)

hypothesis,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
(left) - (front),-0.05120193,0.03515430,-1.4564910,0.1452569,2.783321,-0.12010310,0.01769924
(right) - (front),-0.02459701,0.03526713,-0.6974486,0.4855221,1.042391,-0.09371933,0.04452530
(right) - (left),0.02660491,0.03764641,0.7067052,0.4797497,1.059646,-0.04718069,0.10039052


In [18]:
avg_slopes(model, variables ="Proximity", type = "link")

term,contrast,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high,predicted_lo,predicted_hi,predicted
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Proximity,dY/dX,-0.7219165,0.1929943,-3.74061,0.0001835739,12.41135,-1.100178,-0.3436546,-3.662672,-3.662817,-3.662744


In [19]:
avg_slopes(model, variables ="n_candidates", type = "link")

term,contrast,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high,predicted_lo,predicted_hi,predicted
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
n_candidates,dY/dX,-0.9059379,0.2076671,-4.362454,1.286119e-05,16.24662,-1.312958,-0.498918,-3.662654,-3.662835,-3.662744
